<a href="https://colab.research.google.com/github/alostmathematician/ISTA-366/blob/main/ISTA_322_Sp23_HW2_part_2_qin_xu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  HW2 Part 2 - Flatting JSON data from Yelp

For this second part of the assignment we're going to be using the Yelp API via the python package `yelpapi`.  Like with the Spotify API, you need to go get a yelp developer account here: https://www.yelp.com/fusion.
When you try to create an app, it asks for name, description and contact info. The it gives you clientid and api key.

The goal will be to make a dataset of local taco places that are doing well during Coronavirus. Specifically, we want to make a dataset that takes their average score since they've been open and compares it to the average score of the last three reviews.  Shops that have been doing well should hopefully have these two averages be similar.

There will be three main steps to this process:

* First you'll search by location type and get aggregate information for everything that falls in the ice cream category.
* Second you'll get reviews for all those locations.  Yelp only returns three reviews when you call an ID, but that still works. Given you can only query one ID at a time, you'll need to write a loop to create a dataframe of all the reviews.  
* Third, you'll aggregate the latest review information to see how their average review score compares to their overall average review score.  

In [ ]:
# Mount google drive
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

In [ ]:
# Install yelpapi once. Next time you run this notebook, you can skip this.
!pip install --target=$nb_path yelpapi

In [173]:
# Now enter your API key so you can make requests
from yelpapi import YelpAPI
from pandas import json_normalize
# yelp_api = YelpAPI('ENTER YOUR KEY HERE')
api_key = 'WnBdKzJxJHwtBPuL0_tF2KaUwhMUi_r-6Z4LdCrnT3zFH9-jhQMY5bwFMO3gpS4-5GyGA1L2Be4J3WMyg5vGTfdFFf1FIDk4hLkP-PutWinfhkjFGN8vL8gq54__ZHYx'
yelp_api = YelpAPI(api_key)


### Making calls to with Yelp API

There are many functions in the `yelpapi` package.  The first one we'll use is `search_query()`.  You can put in a term you want to search for followed up by the location and it'll give you all the locations that match.  For example, the following would search for all ice cream places here in Tucson.

```
ice = yelp_api.search_query(term = 'ice cream', location = 'Tucson, AZ')
```

This will give you a response of all the ice cream places in Tucson. It'll be in JSON form so will need flattening before being useful.

### Q6 Getting all taco shops and flattening - [3 points]

**Task:** Start by making a dataframe that uses the `search_query()` function to search using the term 'taco'.  Call this `taco_shops`.  After that, flatten the json results to `taco_shops_df`.

In [174]:
## Q6 part 1 Your code starts here
# search for taco shops and store to taco_shops
taco_shops = yelp_api.search_query(term='taco',  location='Tucson, AZ')

In [175]:
# Look at taco_shops
taco_shops

{'businesses': [{'id': 'jmwasbZfgj3honf79qKsnA',
   'alias': 'street-taco-and-beer-co-tucson',
   'name': 'Street- Taco and Beer Co.',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/VOaRTnYRNSnSF6DR28fUog/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/street-taco-and-beer-co-tucson?adjust_creative=PYOtmAAm6H15Heqs6P7X9w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=PYOtmAAm6H15Heqs6P7X9w',
   'review_count': 936,
   'categories': [{'alias': 'mexican', 'title': 'Mexican'},
    {'alias': 'bars', 'title': 'Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 32.222001, 'longitude': -110.972225},
   'transactions': ['delivery'],
   'price': '$',
   'location': {'address1': '58 W Congress St',
    'address2': None,
    'address3': '',
    'city': 'Tucson',
    'zip_code': '85701',
    'country': 'US',
    'state': 'AZ',
    'display_address': ['58 W Congress St', 'Tucson, AZ 85701']},
   'phone': '+19499492226',
   'display_phone': 

In [176]:
# What keys are present?
taco_shops.keys()
## Q6 part 1  Your code ends here - Any code outside of these start/end markers won't be graded

dict_keys(['businesses', 'total', 'region'])

Now use the json_normalize function to flatten `taco_shops`.  Note that you need to select the key that contains the businesses when flattening.  But this one is easier than the examples from the homework in that you don't need to provide any other arguments.  

Store the result as `taco_shops_df`

After that select only the columns 'id', 'alias', 'name', 'review_count', and 'rating'.

In [177]:
## Q6 part 2 Your code starts here
# Flatten to taco_shops_df
taco_shops_df = json_normalize(taco_shops['businesses'])
taco_shops_df# Check

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,jmwasbZfgj3honf79qKsnA,street-taco-and-beer-co-tucson,Street- Taco and Beer Co.,https://s3-media4.fl.yelpcdn.com/bphoto/VOaRTn...,False,https://www.yelp.com/biz/street-taco-and-beer-...,936,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,[delivery],...,32.222001,-110.972225,58 W Congress St,None,,Tucson,85701,US,AZ,"[58 W Congress St, Tucson, AZ 85701]"
1,W-qxgFKtw8TlibinTp5cYw,momos-tucson,Momo's,https://s3-media1.fl.yelpcdn.com/bphoto/eeytOB...,False,https://www.yelp.com/biz/momos-tucson?adjust_c...,58,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,[],...,32.227550,-110.944140,1838 E 6th St,,None,Tucson,85719,US,AZ,"[1838 E 6th St, Tucson, AZ 85719]"
2,aZ-2J_3nvuT-XdryCjrthg,carniceria-wild-west-tucson-2,Carniceria Wild West,https://s3-media2.fl.yelpcdn.com/bphoto/_oAxTr...,False,https://www.yelp.com/biz/carniceria-wild-west-...,36,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",5.0,"[delivery, pickup]",...,32.294315,-110.961553,4825 N 1st Ave,None,,Tucson,85718,US,AZ,"[4825 N 1st Ave, Tucson, AZ 85718]"
3,Nggy_QUDxaLlrcQAQf7GnQ,taqueria-juanitos-tucson-4,Taqueria Juanito's,https://s3-media4.fl.yelpcdn.com/bphoto/u67QWv...,False,https://www.yelp.com/biz/taqueria-juanitos-tuc...,473,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"[delivery, pickup]",...,32.250571,-110.982543,708 W Grant Rd,,,Tucson,85705,US,AZ,"[708 W Grant Rd, Tucson, AZ 85705]"
4,nvwtUKh6rBb29rFbC9kq5w,taco-rico-tucson-2,Taco Rico,https://s3-media3.fl.yelpcdn.com/bphoto/f9mKwX...,False,https://www.yelp.com/biz/taco-rico-tucson-2?ad...,62,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",5.0,[],...,32.337110,-111.061190,4499 W Ina Rd,None,None,Tucson,85741,US,AZ,"[4499 W Ina Rd, Tucson, AZ 85741]"
5,A-5IN85MwL9F8wJRsDna6g,tacos-apson-tucson,Tacos Apson,https://s3-media2.fl.yelpcdn.com/bphoto/3fm1Nq...,False,https://www.yelp.com/biz/tacos-apson-tucson?ad...,316,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,[],...,32.183140,-110.976883,3501 S 12th Ave,,,Tucson,85713,US,AZ,"[3501 S 12th Ave, Tucson, AZ 85713]"
6,eR44gUajtqCnVECnjh7Ynw,karamelo-king-tucson,Karamelo King,https://s3-media4.fl.yelpcdn.com/bphoto/u9fb8Z...,False,https://www.yelp.com/biz/karamelo-king-tucson?...,91,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,[delivery],...,32.251005,-110.912635,3752 E Grant Rd,,,Tucson,85716,US,AZ,"[3752 E Grant Rd, Tucson, AZ 85716]"
7,nEaTbGFlU7d9eLU2kl6KBw,taqueria-el-pueblito-tucson-2,Taqueria El Pueblito,https://s3-media2.fl.yelpcdn.com/bphoto/EtmIl6...,False,https://www.yelp.com/biz/taqueria-el-pueblito-...,379,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"[delivery, pickup]",...,32.264046,-110.944400,1800 E Fort Lowell Rd,Ste A156,,Tucson,85719,US,AZ,"[1800 E Fort Lowell Rd, Ste A156, Tucson, AZ 8..."
8,wzPc5d1l0PpxQLQDhh8T2w,toro-loco-tacos-and-burros-tucson,Toro Loco Tacos & Burros,https://s3-media1.fl.yelpcdn.com/bphoto/ouVVuA...,False,https://www.yelp.com/biz/toro-loco-tacos-and-b...,215,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.0,"[delivery, pickup]",...,32.236620,-110.870040,5737 E Speedway Blvd,,None,Tucson,85712,US,AZ,"[5737 E Speedway Blvd, Tucson, AZ 85712]"
9,LQcGL4hfJAeK6bk2ZdhmXw,aqui-con-el-nene-tucson,Aqui Con El Nene,https://s3-media3.fl.yelpcdn.com/bphoto/BczDqo...,False,https://www.yelp.com/biz/aqui-con-el-nene-tucs...,398,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"[delivery, pickup]",...,32.286826,-110.995074,4415 N Flowing Wells Rd,,,Tucson,85705,US,AZ,"[4415 N Flowing Wells Rd, Tucson, AZ 85705]"


In [178]:
# Select only necessary columns
taco_shops_df = taco_shops_df[['id', 'alias', 'name', 'review_count','rating']]
taco_shops_df.shape # Check shape.  Is it 20 x 5?
## Q6 part 2  Your code ends here - Any code outside of these start/end markers won't be graded

(20, 5)

## Q7 Getting reviews for the taco shops - [6 points]

Now we're going to use the `reviews_query()` function to get the last three reviews for a given ID.  The issue here is that you can only feed it one ID at a time.  So, we'll have to write a loop that queries for each ID in `taco_shops_df` and builds out a dataframe of reviews.  

**Task:**  Write a for loop that does the following steps:
* First make an empty data frame outside of the loop called `taco_shops_reviews_df`
* Initalize your loop so that it runs the length of `taco_shops_df` you made earlier.
* For each i in loop, use the id from `taco_shops_df` to get reviews from yelp using the `reviews_query()` function in `yelp_api` and store it to an object called `reviews`.
* Flatten `reviews` to an objected called `reviews_df`
* Add `location_id` to `reviews_df` - I gave you the code to do this :)
* Append `reviews_df` to `taco_shops_reviews_df` such that it builds out that dataframe with each `reviews_df` dataframe that's generated each loop.
* After the loop is done select only the columns `'id', 'text', 'rating', 'time_created', 'location_id'`

In [179]:
## Q7 Your code starts here
# Write our loop
import pandas as pd
taco_shops_reviews_df = pd.DataFrame()

for i in range(len(taco_shops_df)):

  reviews = yelp_api.reviews_query(taco_shops_df['id'][i])['reviews']
  reviews_df = json_normalize(reviews)
  reviews_df['location_id'] = taco_shops_df['id'][i]
  taco_shops_reviews_df = pd.concat([taco_shops_reviews_df, reviews_df], ignore_index=True)

In [180]:
# Select columns 'id', 'text', 'rating', 'time_created', 'location_id'
taco_shops_reviews_df = taco_shops_reviews_df[['id','text','rating','time_created','location_id']]
## Q7  Your code ends here - Any code outside of these start/end markers won't be graded

### Q8 Aggregating your review data - [3 points]

**Task:** Now go and do a data aggregation to get the mean review score across the three reviews. Remember, we want this grouped by location_id.  Call this dataframe `latest_reviews_agg`.

In your groupby you should set `as_index` to false to make joining on the ID values easier.

You should also rename the second column to `mean_rating` vs. leaving it as the dual level name.


In [181]:
## Q8 Your code starts here
# Do your groupby to get mean rating.
# Call it latest_reviews_agg.
latest_reviews_agg = taco_shops_reviews_df.groupby(
    'location_id',
    as_index=False).agg(
        {'rating': 'mean'})

In [182]:
# Check it!
latest_reviews_agg

,location_id,rating
0,4r6N_Fhiwoqo_FqQ7Mm6mQ,4.333333
1,A-5IN85MwL9F8wJRsDna6g,4.000000
2,Jcm-2RZWIcF0E4V6kekBsQ,5.000000
3,LQcGL4hfJAeK6bk2ZdhmXw,4.333333
4,MK0OMY_u9unl8xSqjPLtMw,5.000000
5,Nggy_QUDxaLlrcQAQf7GnQ,4.666667
6,RHdEScVIAQ7xzFFMiQEnAQ,4.666667
7,W-qxgFKtw8TlibinTp5cYw,4.666667
8,aZ-2J_3nvuT-XdryCjrthg,5.000000
9,eR44gUajtqCnVECnjh7Ynw,3.666667


In [183]:
# Rename so the two column names are 'location_id' and 'mean_rating'
latest_reviews_agg.columns = ['location_id','mean_rating']
## Q8  Your code ends here - Any code outside of these start/end markers won't be graded

### Q9 Join your two datasets and one last transform - [3 points]

**Task:** Now it's time to join `latest_reviews_agg` back to your `taco_shops_df` dataframe.  You're going to want to join them on the location_id, but remember it is called just 'id' in the `taco_shops_df`  dataframe.  Call this joined dataset `taco_shops_comp`

After you make that dataset do one last transform.  In this I want you to make a new column called 'still_good' where the value is 'yes' if the mean_rating is greater than or equal to the average rating since they opened, or 'no' if the rating has dropped.   The idea here is that this could be a value that one would use to see if their average score of the latest reviews has improved or suffered during Covid.  

In [184]:
## Q9 Your code starts here
# Join and name taco_shops_comp
taco_shops_comp = taco_shops_df.merge(latest_reviews_agg,how='left',left_on='id',right_on='location_id')

When you are comparing the columns make sure to not comparing taco_shops_df and taco_shops_comp since their tuples may have different orders! Use both columns from taco_shops_comp

In [185]:
# Make still_good column
taco_shops_comp['still_good'] = taco_shops_comp['mean_rating'] >= taco_shops_comp['rating']
taco_shops_comp['still_good'] = taco_shops_comp['still_good'].map({True: 'yes', False: 'no'})


In [186]:
taco_shops_comp

,id,alias,name,review_count,rating,location_id,mean_rating,still_good
0,jmwasbZfgj3honf79qKsnA,street-taco-and-beer-co-tucson,Street- Taco and Beer Co.,936,4.5,jmwasbZfgj3honf79qKsnA,4.666667,yes
1,W-qxgFKtw8TlibinTp5cYw,momos-tucson,Momo's,58,4.5,W-qxgFKtw8TlibinTp5cYw,4.666667,yes
2,aZ-2J_3nvuT-XdryCjrthg,carniceria-wild-west-tucson-2,Carniceria Wild West,36,5.0,aZ-2J_3nvuT-XdryCjrthg,5.000000,yes
3,Nggy_QUDxaLlrcQAQf7GnQ,taqueria-juanitos-tucson-4,Taqueria Juanito's,473,4.5,Nggy_QUDxaLlrcQAQf7GnQ,4.666667,yes
4,nvwtUKh6rBb29rFbC9kq5w,taco-rico-tucson-2,Taco Rico,62,5.0,nvwtUKh6rBb29rFbC9kq5w,5.000000,yes
5,A-5IN85MwL9F8wJRsDna6g,tacos-apson-tucson,Tacos Apson,316,4.0,A-5IN85MwL9F8wJRsDna6g,4.000000,yes
6,eR44gUajtqCnVECnjh7Ynw,karamelo-king-tucson,Karamelo King,91,4.5,eR44gUajtqCnVECnjh7Ynw,3.666667,no
7,nEaTbGFlU7d9eLU2kl6KBw,taqueria-el-pueblito-tucson-2,Taqueria El Pueblito,379,4.0,nEaTbGFlU7d9eLU2kl6KBw,4.333333,yes
8,wzPc5d1l0PpxQLQDhh8T2w,toro-loco-tacos-and-burros-tucson,Toro Loco Tacos & Burros,215,4.0,wzPc5d1l0PpxQLQDhh8T2w,4.333333,yes
9,LQcGL4hfJAeK6bk2ZdhmXw,aqui-con-el-nene-tucson,Aqui Con El Nene,398,4.5,LQcGL4hfJAeK6bk2ZdhmXw,4.333333,no


In [187]:
# So, do any locations have a lower average in their last three reviews compared to their average overall score?
last_three_lower = taco_shops_comp[taco_shops_comp['still_good'] == 'no']
last_three_lower
## Q9  Your code ends here - Any code outside of these start/end markers won't be graded

,id,alias,name,review_count,rating,location_id,mean_rating,still_good
6,eR44gUajtqCnVECnjh7Ynw,karamelo-king-tucson,Karamelo King,91,4.5,eR44gUajtqCnVECnjh7Ynw,3.666667,no
9,LQcGL4hfJAeK6bk2ZdhmXw,aqui-con-el-nene-tucson,Aqui Con El Nene,398,4.5,LQcGL4hfJAeK6bk2ZdhmXw,4.333333,no
13,jD4hInLGaQgSJfidMf1T4A,polos-taqueria-tucson-3,Polos Taqueria,4,4.5,jD4hInLGaQgSJfidMf1T4A,4.333333,no
18,otQS34_MymijPTdNBoBdCw,bk-tacos-tucson,BK Tacos,570,4.0,otQS34_MymijPTdNBoBdCw,3.666667,no
19,orLNj9bDRMxaz6Hgs9YvVg,agave-house-tucson-2,Agave House,40,4.5,orLNj9bDRMxaz6Hgs9YvVg,3.666667,no


You should get 7 shops (this may change suddenly since data is live, but on Feb 8th is still 7) including Street- Taco and Beer Co. and Street- Taco and Beer Co. unless it gets 3 five star reviews until the submission deadline :)